In [ ]:
from Bio import Entrez
import csv

def pubmed_scraper(keyword, start_date, end_date, filename):
    #set email address for NCBI id
    Entrez.email = "isabela.baker@uth.tmc.edu"
    
    #define search
    search_query = f"{keyword} AND ({start_date}:{end_date}[Date - Publication])"
    
    #perform search 
    handle = Entrez.esearch(db="pubmed", term=search_query, retmode="xml")
    record = Entrez.read(handle)
    handle.close()

    #fetch paper details 
    papers = []
    for id in record["IdList"]:
        pubmed_article = Entrez.efetch(db="pubmed", id=id, retmode="xml")
        pubmed_record = Entrez.read(pubmed_article)[0]
        pubmed_article.close()

        title = pubmed_record[0]["MedlineCitation"]["Article"]["ArticleTitle"]
        authors = ", ".join([author["LastName"] + " " + author["Initials"] for author in pubmed_record["MedlineCitation"]["Article"]["AuthorList"]])
        publication_time = pubmed_record["MedlineCitation"]["Article"]["ArticleDate"]
        abstract = pubmed_record[0]["MedlineCitation"]["Article"]["Abstract"]["AbstractText"][0] if "Abstract" in pubmed_record["MedlineCitation"]["Article"] else ""
        mesh = pubmed_record[0]["MedlineCitation"]["MeshHeadingList"]
        descriptors = ','.join(term["DescriptorName"] for term in mesh)

        papers.append([title, authors, publication_time, abstract, descriptors])

    #save data to csv
    with open(filename, "w", newline="", encoding="utf-8") as csvfile:
        csvwriter = csv.writer(csvfile)
        csvwriter.writerow(["Title", "Authors", "publication Time", "Abstract"])
        csvwriter.writerows(papers)

if __name__ == "__main__":
    keyword = "HIV"
    start_date = "2020/01/01"
    end_date = "2020/08/30"
    filename = "pubmed_data.csv"

    pubmed_scraper(keyword, start_date, end_date, filename) 